In [49]:
import pandas as pd
import numpy as np

In [50]:
TRAINING_DATA_FILE = 'C:/Users/rachi/Downloads/Model4/train-data.txt'
TEST_DATA_FILE = 'C:/Users/rachi/Downloads/Model4/test-data.txt'

VOCAB_SIZE = 2500

In [51]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype = int)

In [52]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter = ' ', dtype = int)

In [53]:
sparse_train_data[:5]

array([[ 1,  0,  0,  1],
       [ 1,  1,  0, 17],
       [ 1,  4,  0,  2],
       [ 1,  5,  0, 10],
       [ 1,  6,  0,  1]])

In [54]:
sparse_train_data[-5:]

array([[9999, 1466,    1,    1],
       [9999, 1751,    1,    1],
       [9999, 1773,    1,    1],
       [9999, 1931,    1,    1],
       [9999, 1945,    1,    1]])

In [55]:
print('Number of rows in training file: ', sparse_train_data.shape[0])
print('Number of rows in test file: ', sparse_test_data.shape[0])

Number of rows in training file:  382378
Number of rows in test file:  163300


In [56]:
print('Number of emails in training file:', np.unique(sparse_train_data[:,0]).size)

Number of emails in training file: 6981


In [57]:
print('Number of emails in test file:', np.unique(sparse_test_data[:,0]).size)

Number of emails in test file: 2992


In [58]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [59]:
len(column_names)

2502

In [60]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   1,    2,    4, ..., 9996, 9997, 9999])

In [61]:
full_train_data = pd.DataFrame(index = index_names, columns = column_names)
full_train_data.fillna(value=0, inplace = True)

C:\Users\rachi\AppData\Local\Temp\ipykernel_9820\2370846799.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_train_data.fillna(value=0, inplace = True)


In [62]:
full_train_data.head()

DOC_ID  CATEGORY  0  1  2  3  4  5  6  7  ...  2490  2491  2492  2493  \
1       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
4       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
5       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
6       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

   2494  2495  2496  2497  2498  2499  
1     0     0     0     0     0     0  
2     0     0     0     0     0     0  
4     0     0     0     0     0     0  
5     0     0     0     0     0     0  
6     0     0     0     0     0     0  

[5 rows x 2502 columns]

In [63]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value=0, inplace = True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace = True)
    return full_matrix

In [64]:
%%time

full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

C:\Users\rachi\AppData\Local\Temp\ipykernel_9820\630567689.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace = True)


CPU times: total: 19.3 s
Wall time: 23.8 s


In [65]:
full_train_data.head()

CATEGORY  0   1  2  3  4   5  6  7  8  ...  2490  2491  2492  2493  \
DOC_ID                                         ...                           
1              0  1  17  0  0  2  10  1  1  0  ...     0     0     0     0   
2              0  0   0  0  0  0   0  0  0  0  ...     0     0     0     0   
4              0  0  14  0  0  1   8  0  0  0  ...     0     0     0     0   
5              0  0   0  0  0  1   0  0  0  0  ...     0     0     0     0   
6              0  0   1  0  0  0   0  0  0  0  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
1          0     0     0     0     0     0  
2          0     0     0     0     0     0  
4          0     0     0     0     0     0  
5          0     0     0     0     0     0  
6          0     0     0     0     0     0  

[5 rows x 2501 columns]

In [66]:
full_train_data.tail()

CATEGORY  0  1  2  3  4  5  6  7  8  ...  2490  2491  2492  2493  \
DOC_ID                                       ...                           
9994           1  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
9995           1  0  0  0  1  0  0  0  0  0  ...     0     0     0     0   
9996           1  0  0  2  2  2  0  0  0  0  ...     1     0     0     0   
9997           1  0  0  0  1  0  0  0  1  0  ...     0     0     0     0   
9999           1  0  0  2  0  0  0  0  0  1  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
9994       0     0     0     0     0     0  
9995       0     0     0     0     0     0  
9996       0     0     0     0     0     0  
9997       0     0     0     0     0     0  
9999       0     0     0     0     0     0  

[5 rows x 2501 columns]

In [67]:
# Step: Import required models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Step: Define base models
nb_model = MultinomialNB()
lr_model = LogisticRegression(solver='liblinear')  # good for small datasets
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)  # lower trees = less memory


In [68]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# import joblib
# Assuming models are already trained
joblib.dump(nb_model, 'naive_bayes_model.pkl')
joblib.dump(lr_model, 'logistic_regression_model.pkl')
joblib.dump(rf_model, 'random_forest_model.pkl')


['random_forest_model.pkl']

In [69]:
# y_train = pd.read_csv('y_train.csv')  # or however your labels are stored
y_train = y_train[:full_train_data.shape[0]]


In [70]:
full_train_data.columns = full_train_data.columns.astype(str)


In [71]:
# Fit the models
nb_model.fit(full_train_data.values, y_train)
lr_model.fit(full_train_data.values, y_train)
rf_model.fit(full_train_data.values, y_train)

# Now make predictions
nb_preds = nb_model.predict(full_train_data)
lr_preds = lr_model.predict(full_train_data)
rf_preds = rf_model.predict(full_train_data)


C:\Users\rachi\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rachi\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rachi\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\rachi\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [72]:
# Stack base model predictions as features for the meta-model
stacked_train = np.column_stack((nb_preds, lr_preds, rf_preds))


In [73]:
y_train_small = y_train[:stacked_train.shape[0]]  # Ensure it matches the number of samples in stacked_train


In [74]:
# Train the final stacking model (meta-model)
meta_model = LogisticRegression()
meta_model.fit(stacked_train, y_train_small)


C:\Users\rachi\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [75]:
import joblib
joblib.dump(meta_model, 'meta_model.pkl')


['meta_model.pkl']

In [76]:
sparse_test_data.shape

(163300, 4)

In [77]:
%%time

full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)


C:\Users\rachi\AppData\Local\Temp\ipykernel_9820\630567689.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace = True)


CPU times: total: 10.4 s
Wall time: 12 s


In [78]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [79]:
TEST_FEATURE_MATRIX = 'test-features.txt'
TEST_TARGET_FILE = 'test-target.txt'

np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)

In [81]:
X_train = pd.read_csv('X_train.csv')

C:\Users\rachi\AppData\Local\Temp\ipykernel_9820\719296962.py:1: DtypeWarning: Columns (767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,

In [85]:
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Define base learners
base_models = [
    ('nb', MultinomialNB()),
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('lr', LogisticRegression(max_iter=1000))
]

# Define full stacking model
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),
    passthrough=False
)

# Train it
stacking_model.fit(X_train, y_train)

# Save full stacking model
import joblib
joblib.dump(stacking_model, 'stacking_model.pkl')



C:\Users\rachi\AppData\Roaming\Python\Python312\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rachi\AppData\Roaming\Python\Python312\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ValueError: could not convert string to float: 'dear'